In [3]:
!pip install -q flwr tensorflow pandas scikit-learn


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Cardio dataset
cardio_df = pd.read_csv("/content/cardio_train.csv", sep=';')
X_cardio = cardio_df.drop(columns=["cardio"])
y_cardio = cardio_df["cardio"]
X_cardio = StandardScaler().fit_transform(X_cardio)
Xc_train, Xc_test, yc_train, yc_test = train_test_split(X_cardio, y_cardio, test_size=0.2)

# Breast cancer dataset
cancer_df = pd.read_csv("/content/breast-cancer.csv")
cancer_df["diagnosis"] = cancer_df["diagnosis"].map({"M": 1, "B": 0})
cancer_df = cancer_df.drop(columns=["id"])
X_cancer = cancer_df.drop(columns=["diagnosis"])
y_cancer = cancer_df["diagnosis"]
X_cancer = StandardScaler().fit_transform(X_cancer)
Xb_train, Xb_test, yb_train, yb_test = train_test_split(X_cancer, y_cancer, test_size=0.2)


In [4]:
import tensorflow as tf

def create_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(16, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [5]:
import flwr as fl

class FederatedClient(fl.client.NumPyClient):
    def __init__(self, X_train, y_train, X_test, y_test, input_dim):
        self.model = create_model(input_dim)
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test

    def get_parameters(self, config=None):
        return self.model.get_weights()

    def fit(self, parameters, config=None):
        self.model.set_weights(parameters)
        self.model.fit(self.X_train, self.y_train, epochs=1, batch_size=32, verbose=0)
        return self.model.get_weights(), len(self.X_train), {}

    def evaluate(self, parameters, config=None):
        self.model.set_weights(parameters)
        loss, acc = self.model.evaluate(self.X_test, self.y_test, verbose=0)
        return loss, len(self.X_test), {"accuracy": acc}


In [6]:
from flwr.server.strategy import FedAvg
from flwr.server import ServerConfig, start_server

# Start server
def start_flower_server():
    strategy = FedAvg(min_available_clients=2)
    config = ServerConfig(num_rounds=3)
    start_server(server_address="localhost:8081", config=config, strategy=strategy) # Changed port to 8081

In [7]:
def start_client_1():
    client1 = FederatedClient(Xc_train, yc_train, Xc_test, yc_test, Xc_train.shape[1])
    fl.client.start_numpy_client(server_address="localhost:8081", client=client1) # Changed port to 8081

def start_client_2():
    client2 = FederatedClient(Xb_train, yb_train, Xb_test, yb_test, Xb_train.shape[1])
    fl.client.start_numpy_client(server_address="localhost:8081", client=client2) # Changed port to 8081

In [8]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Load datasets
cardio_df = pd.read_csv("/content/cardio_train.csv", sep=";")
cancer_df = pd.read_csv("/content/breast-cancer.csv")

# ==============================
# Preprocess Cancer Dataset
# ==============================

# Drop ID column and encode diagnosis
cancer_df = cancer_df.drop(columns=["id"])
cancer_df["diagnosis"] = LabelEncoder().fit_transform(cancer_df["diagnosis"])  # M=1, B=0

X_cancer = cancer_df.drop(columns=["diagnosis"])
y_cancer = cancer_df["diagnosis"]

# Train-test split and scale
X_cancer_train, X_cancer_test, y_cancer_train, y_cancer_test = train_test_split(X_cancer, y_cancer, test_size=0.2, random_state=42)
scaler_cancer = StandardScaler()
X_cancer_train = scaler_cancer.fit_transform(X_cancer_train)
X_cancer_test = scaler_cancer.transform(X_cancer_test)

# Train model
model_cancer = LogisticRegression(max_iter=1000)
model_cancer.fit(X_cancer_train, y_cancer_train)
preds_cancer = model_cancer.predict(X_cancer_test)
acc_cancer = accuracy_score(y_cancer_test, preds_cancer)

# ==============================
# Preprocess Cardio Dataset
# ==============================

# Drop irrelevant columns (if any)
cardio_df = cardio_df.drop(columns=["id", "gender"], errors="ignore")  # drop only if present

X_cardio = cardio_df.drop(columns=["cardio"])
y_cardio = cardio_df["cardio"]

# Train-test split and scale
X_cardio_train, X_cardio_test, y_cardio_train, y_cardio_test = train_test_split(X_cardio, y_cardio, test_size=0.2, random_state=42)
scaler_cardio = StandardScaler()
X_cardio_train = scaler_cardio.fit_transform(X_cardio_train)
X_cardio_test = scaler_cardio.transform(X_cardio_test)

# Train model
model_cardio = LogisticRegression(max_iter=1000)
model_cardio.fit(X_cardio_train, y_cardio_train)
preds_cardio = model_cardio.predict(X_cardio_test)
acc_cardio = accuracy_score(y_cardio_test, preds_cardio)

# ==============================
# Results
# ==============================

print(f"Cancer Dataset Accuracy: {acc_cancer:.4f}")
print(f"Cardiovascular Dataset Accuracy: {acc_cardio:.4f}")


Cancer Dataset Accuracy: 0.9737
Cardiovascular Dataset Accuracy: 0.7238
